# Paillier's encryption Scheme

$$
\alpha +_E \beta = E(\alpha + \beta \ mod\ N)\\
x\times_E \alpha = E(x\alpha \ mod \ N)\\
\oplus_{i=1}^{t+1}a_i = a_1 +_E a_2 +_E \cdots +_E a_{t+1}
$$


### Key Generation:

Generate two large primes $P, Q$ of equal length.

and set $N=PQ$. Let $\lambda(N)=lcm(P-1, Q-1)$ be the Carmichael function of N. 

Finally choose $\Gamma \in Z^*_{N^2}$ such that its orders is a multiple of $N$.

The public key is $(N, \Gamma)$ and the secret key is $\lambda(N)$

In [49]:
from klefki.numbers import carmichael, lcm

In [52]:
from klefki.numbers import length as dlen
import random
import math

from klefki.types.algebra.concrete import (
    EllipticCurveGroupSecp256k1 as ECG,
    EllipticCurveCyclicSubgroupSecp256k1 as CG,
    FiniteFieldSecp256k1 as F,
    FiniteFieldCyclicSecp256k1 as CF
)


In [53]:
P, Q = CF.P, CG.N

dlen(P) == dlen(Q)
N = P*Q

In [54]:
lam_N = lcm(P-1, Q-1)

In [55]:
G = random.randint(0, N**2)

In [56]:
print("PubKey: %s; %s" % (N, G))

PubKey: 13407807929942597099574024998205846127379224100613902121136927097058285002635891330411377376978090146667648480129683279260917149325652956599247552883069569; 178975496142136678886098344801609975410614022722901894323102170929250265219633021984111382641038952426922984325023027515693868588779994700407704024078755041519207525646575882306242842887378176508411942903373439348546987659691614926547486099885346412697475195924563350065889766520545825356944779744042693448974


In [57]:
print("PrivKey: %s" % lam_N)

PrivKey: 115792089237316195423570985008687907852837564279074904382605163141518161494336


### Encryption:

To Encrypt a message $m \in Z_N$, select $x \in_R Z_N^*$ and return $c=\Gamma^mx^N\ mod \ N^2$.

In [126]:
m = 7
assert m < N

In [127]:
x = random.randint(0, N)

In [128]:
c = (pow(G, m, N) * pow(x, N, N)) % (N*N)

In [129]:
print("Ciphtertext Text: %s" % c)

Ciphtertext Text: 8517442062247805905990950490466898585343192524256276941684817661260258183533830711392595319469656361655761549470179308537579763659103791278944445126025910350324544604065585664908463387246846239746291730024121889920218584413896103769524662595782971295952127197304460143216199812672041674001033384893774655168


### Decryption:

To decrypt a ciphertext $c \in Z_N$, let $L$ be a function defined over the set $\{u\in Z_{N^2}: u=1 \ mod \ N\}$ computed as $L(u) = (u-1)/N$. Then the decryption of $c$ is computed as

$$
\frac{L(c^{\lambda(N)})}{L(\Gamma^{\lambda(N)})}\ mod\ N
$$

In [130]:
L = lambda u: (u - 1) / N

In [131]:
(L(pow(c, lam_N, N)) / L(pow(G, lam_N, N))) % N

7.0

## Ref

* Wikipedia: Pillier Cryptosystem https://en.wikipedia.org/wiki/Paillier_cryptosystem


* How to Secretly Share the Treasure Map of the Captain? http://www.lirmm.fr/~imbert/jsec/Naveed_ISLAM_poster_SPIE_1.1.pdf


* Encryption Performance Improvementsof the Paillier Cryptosystem https://eprint.iacr.org/2015/864.pdf


* Stackoverflow - Paillier algorithm encryption https://stackoverflow.com/questions/29217630/paillier-algorithm-encryption